In [2]:
import autogen
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [3]:
config_list = {
    'model': 'llama3.1',
    'base_url': 'http://localhost:11434/v1',
    'api_key': 'ollama',
}

In [9]:
prompt = input("What do you want to generate today? ")
prompt

'I want to generate a story within 100 words'

In [5]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
input_string = prompt
tokens = word_tokenize(input_string)
prompt = " ".join([lemmatizer.lemmatize(token, pos='v') 
                                  for token in tokens 
                                  if token.lower() not in stop_words and token not in punctuation])
print("Original Tokens:", tokens)
print("Lemmatized Tokens:", prompt)


Original Tokens: ['I', 'want', 'to', 'generate', 'a', 'story', 'within', '100', 'words']
Lemmatized Tokens: want generate story within 100 word


In [6]:
text_assistant = autogen.ConversableAgent(
    'assistant', 
    system_message=f'I am here to intiate the chat',
    llm_config=config_list, 
    human_input_mode='NEVER',
    code_execution_config=False
)

text_assistant1 = autogen.ConversableAgent(
    'assistant', 
    system_message=f'{prompt}',
    llm_config=config_list, 
    human_input_mode='NEVER',
    code_execution_config=False
)

evaluation_agent_1 = autogen.ConversableAgent(
    'evaluation-agent-1',
    system_message=f'You have to validate if the generated content is according to the {prompt}. Just reply with "validated" or "not validated".',
    llm_config=config_list,
    human_input_mode='NEVER',
    code_execution_config=False,
)

evaluation_agent_2 = autogen.ConversableAgent(
    'evaluation-agent-2',
    system_message='Rate the content on a scale of 1 to 10 for clarity, accuracy, and creativity.',
    llm_config=config_list,
    human_input_mode='NEVER',
    code_execution_config=False,
)

In [7]:
conversation_state = 1
speakers = []
conversation_log=[]

print(f"Current conversation state: {conversation_state}")


def content_generation_func(speaker: autogen.AssistantAgent, groupchat: autogen.GroupChat):
    global conversation_state
    
    print(f"Current speaker: {speaker.name}, Conversation state: {conversation_state}")
    conversation_log.append({'agent': speaker.name, 'message': "Message from " + speaker.name})
    
    if conversation_state == 1:
        if speaker == text_assistant:
            
            conversation_state += 1
            speakers.append(text_assistant1.name)
            return text_assistant1  
        
    
    elif conversation_state == 2:
        if speaker == text_assistant1:
            
            conversation_state += 1
            speakers.append(evaluation_agent_1.name)
            return evaluation_agent_1  
    
    elif conversation_state == 3:
        if speaker == evaluation_agent_1:
            
            speakers.append(evaluation_agent_2.name)
            print("Would you like to generate the story again? (yes or no)")
            user_input = input().strip().lower()  

            if user_input == 'yes':
                print("Regenerating the story...")
                conversation_state = 1
                speakers.clear()  
                return text_assistant  
            else:
                print("Thank you! Ending the conversation.")
                conversation_state += 1
                return evaluation_agent_2  

        
    elif conversation_state == 4:
        if speaker == evaluation_agent_1:
           conversation_state += 1
           return None  

    else:
        print("Unexpected state or speaker.")
        return None


Current conversation state: 1


In [8]:
content_generation_chat = autogen.GroupChat(
    agents=[text_assistant,text_assistant1, evaluation_agent_1, evaluation_agent_2],
    speaker_selection_method=content_generation_func,
    messages=[],  
    max_round=4
)

content_generation_manager = autogen.GroupChatManager(
    groupchat=content_generation_chat, llm_config=config_list
)

user_message = f"{prompt}"

generated_content = text_assistant.initiate_chat(
    content_generation_manager,
    message=user_message
)

assistant (to chat_manager):

want generate story within 100 word

--------------------------------------------------------------------------------
Current speaker: assistant, Conversation state: 1

Next speaker: assistant

assistant (to chat_manager):

Here's a 100-word story:

As the sun set over the ocean, a young girl named Luna gathered seashells on the beach. She loved listening to her grandmother's stories about mermaids and sea sirens that sang in harmony with the waves. As she walked along the shoreline, Luna stumbled upon a shimmering shell with a note attached to it. The note read: "For Luna, follow the tide." She looked out at the water and saw a glimpse of turquoise. Without hesitation, she dove into the ocean, following the tide that whispered secrets only she could hear.

(Finished!)

--------------------------------------------------------------------------------
Current speaker: assistant, Conversation state: 2

Next speaker: evaluation-agent-1

evaluation-agent-1 (to 